In [1]:
#MAKE SURE MAX(the app) IS OPEN AND RUNNING BEFORE RUNNING THIS NOTEBOOK!!!

In [2]:
import quantfinal
#%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *

import random
import time
from pythonosc.udp_client import SimpleUDPClient
import maxpy as mp
import subprocess

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi
import numpy as np

In [3]:
"""
Variables to be changed in each instance. 
"""

from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from numpy import pi

qreg_q = QuantumRegister(4, 'q')
creg_c = ClassicalRegister(4, 'c')
qc = QuantumCircuit(qreg_q, creg_c)

qc.h(qreg_q[0])
qc.rx(pi / 2, qreg_q[3])
qc.cx(qreg_q[0], qreg_q[1])
qc.rxx(pi / 2, qreg_q[2], qreg_q[3])
qc.cx(qreg_q[1], qreg_q[2])
qc.cx(qreg_q[2], qreg_q[3])

qc.measure([0, 1, 2, 3], [0, 1, 2, 3])

qc.draw()

#number of components
num_components = 6


#expected probabilities of each state
expected_probabilities = [12.5, 0, 0, 12.5, 12.5, 0, 0, 12.5, 
                          12.5, 0, 0, 12.5, 12.5, 0, 0, 12.5]

In [4]:
"""
Variables to stay the same, for now (could change later).
"""
shots = 1024
num_qbits = 4
num_states = 16

assert(len(expected_probabilities) == num_states)

waiting_ambient_length = 4. #time between each ambient note for waiting section (s)
waiting_ambient_slide_f = 100. #slide up to freq from -100
waiting_ambient_slide_time = 1000. #slide up time (ms)

states_bits = ["0000", "0001", "0010", "0011", 
               "0100", "0101", "0110", "0111",
               "1000", "1001", "1010", "1011",
               "1100", "1101", "1110", "1111"]

states = [[], ['gran'], ['glitch'], ['glitch', 'gran'],
          ['gloop'], ['gloop', 'gran'], ['gloop', 'glitch'], ['gloop', 'glitch', 'gran'],
          ['back'], ['back', 'gran'], ['back', 'glitch'], ['back', 'gran', 'glitch'],
          ['back', 'gloop'], ['back', 'gloop', 'gran'], ['back', 'gloop', 'glitch'], 
          ['back', 'gloop', 'glitch', 'gran']]

on_corresponding_states = {'gran': "0001", 'glitch': "0010", 'back': "1000", 'gloop': "0100"}

In [5]:
"""
Variables that MUST stay the same!!!!
"""
ambient_synth_name = "ambient"
amb_sequencer_name = "ambient-seq"
over_sequencer_name = "over-seq"

## First, build the MaxMSP patch & set up waiting music!

In [6]:
"""
Set up OSC port. 
"""
ip = "127.0.0.1"
port = 5005
client = SimpleUDPClient(ip, port)

In [7]:
"""
Generate frequencies, ambient color, and waiting music sequence.
"""

#frequencies
freqs = quantfinal.make_freqs(num_components, num_states)
random.shuffle(freqs)

#ambient and oversound waiting sequences 
ambient_seq, over_seq = quantfinal.get_waiting_seq(freqs, expected_probabilities, states, 
                                                  waiting_ambient_slide_f, waiting_ambient_slide_time,
                                                  waiting_ambient_length)

#color
color = quantfinal.get_color(num_components)

In [8]:
"""
Build ambient synth, ambient sequencer, and oversound sequencer. 
"""

#build ambient synth
quantfinal.build_ambient_synth(num_components, ambient_synth_name)

#build ambient sequencer
quantfinal.create_amb_seq_patch(ambient_seq, amb_sequencer_name)

#build oversound sequencer
quantfinal.create_over_seq_patch(over_seq, over_sequencer_name)

ObjectError: pan2 : creation : object unknown
ObjectError: pan2 : creation : object unknown
ObjectError: pan2 : creation : object unknown
ObjectError: pan2 : creation : object unknown
ObjectError: pan2 : creation : object unknown
ObjectError: pan2 : creation : object unknown
  ObjectMsg: ambient-bottom : creation : ambient-bottom.maxpat file found, abstraction created
(['osc-to-add-0'],)
['osc-to-add-0']
  ObjectMsg: osc-to-add-0 : creation : osc-to-add-0.maxpat file found, abstraction created
(['osc-to-add-1'],)
['osc-to-add-1']
  ObjectMsg: osc-to-add-1 : creation : osc-to-add-1.maxpat file found, abstraction created
(['osc-to-add-2'],)
['osc-to-add-2']
  ObjectMsg: osc-to-add-2 : creation : osc-to-add-2.maxpat file found, abstraction created
(['osc-to-add-3'],)
['osc-to-add-3']
  ObjectMsg: osc-to-add-3 : creation : osc-to-add-3.maxpat file found, abstraction created
(['osc-to-add-4'],)
['osc-to-add-4']
  ObjectMsg: osc-to-add-4 : creation : osc-to-add-4.maxpat file found, abstracti

In [9]:
"""
Open maxpatch and send color. 
"""

subprocess.call(['open', 'quantum-patch.maxpat'])
time.sleep(5)
client.send_message("/ambient-color", color)

In [10]:
"""
Now, start the waiting music!
"""
client.send_message("/amb-seq-toggle", 1)
client.send_message("/over-seq-toggle", 1)

## Then, run the quantum circuit stuff!

In [11]:
"""
Setting up IBM machine.
"""
# Loading your IBM Q account(s)
IBMQ.save_account("ffa8bc9d57f1abc38c534ea5cbc36a23e864ffb7d067e7c504be089e60a78ea7a058d2a5ce8b7f30ebf3c357ef288e2a44cfc2f2a84cff9e302ec1b0e93571e7")
provider = IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')



configrc.store_credentials:WARNING:2022-10-21 17:58:40,663: Credentials already present. Set overwrite=True to overwrite.


In [12]:
"""
Give the waiting music a little time to breathe....
"""

time.sleep(9)

In [13]:
"""
Run the quantum circuit!
"""
    
#use a real backend
backend = provider.get_backend('ibmq_qasm_simulator')

#check if this backend supports memory
if backend.configuration().memory :
    print(backend.name() + " supports memory to see invidual results.")
else:
    print(backend.name() + " does not support individual results... defaulting to ourense")

job = execute(qc, backend, shots=shots, memory=True)

#we want to see results independently, instead of as a probability
result = job.result()
memory = result.get_memory()

# need an array to drop all the results into
outputArray = []

#convert results to int and drop into array
for x in range(0, shots):
#     converted = int(memory[x], 2)
    outputArray.append(memory[x])

from collections import defaultdict

measured_probs = defaultdict(lambda: 0)
for key, val in job.result().get_counts().items():
    measured_probs[key] = val

ibmq_qasm_simulator supports memory to see invidual results.


## Now, quantum music time!!!

(you can add any file saving / TD needed stuff here!)

In [14]:
"""
First, turn off the sequencers.
"""
client.send_message("/amb-seq-toggle", 0)
client.send_message("/over-seq-toggle", 0)

In [15]:
import json

with open("test.json", "w") as outfile:
    json.dump(outputArray, outfile)

In [16]:
"""
Then, start feeding the measured states into Max!
"""

freq_slide = waiting_ambient_slide_f
freq_slide_time = waiting_ambient_slide_time
ambient_time = waiting_ambient_length

for i in range(len(outputArray)):
    measured_state = outputArray[i]
    state_index = states_bits.index(measured_state)
    freq = freqs[state_index]
    
    #calculate whether or not that state's overnoises would show up, using probabilities
    over_state = states[state_index]
    
    oversounds_to_play = []
    for oversound in over_state:
        corresponding_state = on_corresponding_states[oversound]
        oversound_prob = measured_probs[corresponding_state] / shots
        play = np.random.choice([0, 1], p=(oversound_prob, 1-oversound_prob))
        if play:
            oversounds_to_play.append(oversound)
    
    num_oversounds = len(oversounds_to_play)
    if num_oversounds != 0:
        time_per_oversound = ambient_time / num_oversounds
    
    #send ambient
    client.send_message("/ambient-freq", [freq-freq_slide, freq, freq_slide_time])
    client.send_message("/ambient", 1)
    if num_oversounds !=0: 
        #for multiple oversounds
        for sound in oversounds_to_play:
            if oversounds_to_play[0] == "gloop":
                client.send_message("/gloop-freq", freq)
        client.send_message("/" + oversound, 1)
        time.sleep(time_per_oversound)
    else:
        time.sleep(ambient_time)
    
    if freq_slide < 200:
        freq_slide += 10
    
    if freq_slide_time > 500:
        freq_slide_time -= 50
        
    if ambient_time > 1.:
        ambient_time -= 0.3
        
    if i%10 == 0:
        print(i)
        
        
    #send corresponding state frequency to ambient 
    #play overnoises (depending on probability), also within ambient time 
    #slowly decrease ambient time? 
    #slowly increase slide
    #slowly decrease slide time

0
10
20
30


KeyboardInterrupt: 